In [2]:
import pandas as pd
from scipy.stats import truncnorm
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.special import expit as activation_function

In [3]:
df = pd.read_csv("archive/fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df = np.array(df)
m, n = df.shape
np.random.shuffle(df)

#data_dev = df[0:1000].T
#Y_data = data_dev[0]
#X_data = data_dev[1:n]


data_train = df[1000:m].T
Y_train = data_train[0] 
X_train = data_train[1:n]
X_train = X_train.apply(lambda x: (x - x.min(axis = 0)) (x.max(axis = 0) - x.min(axis = 0)))
#X_train

AttributeError: 'numpy.ndarray' object has no attribute 'apply'

In [4]:
print(data_train[0])

[4 1 0 ... 2 3 8]


In [5]:
X_train[:,0].shape

(784,)

In [10]:
def init_params():
    w1 = np.random.rand(10, 784) - 0.5 
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5 
    b2 = np.random.rand(10, 1) - 0.5
    return w1, b1, w2, b2

def ReLU(z):
    return np.maximum(0,z)

def softmax(z):
    return np.exp(z) / np.sum(np.exp(z), axis = 0)

def forward_prop(w1, b1, w2, b2, X):
    z1 = w1.dot(X) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2
    
def one_hot(Y):
    one_hot_Y = np.zeros ( (Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def derivative_ReLU(z):
    return z > 0
    
def back_prop(z1, a1, w2, z2, a2, Y, X):
    #m = len(Y)
    one_hot_Y = one_hot(Y)
    dz2 = a2 - one_hot_Y
    dw2 = 1/m * dz2.dot(a1.T)
    db2 = 1/m * np.sum(dz2)
    dz1 = w2.T.dot(dz2) * derivative_ReLU(z1)
    dw1 = 1/m * dz1.dot(X.T)
    db1 = 1/m * np.sum(dz1)
    return dw1, db1, dw2, db2

#def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1
    w2 = w2 - alpha * dw2
    b2 = b2 - alpha * db2
    return w1, b1, w2, b2
    
def get_predictions (a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X,Y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range (iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, X)
        dw1, db1, dw2, db2 = back_prop (z1, a1, w2, z2, a2, Y, X)
        w1, b1, w2, b2 = update_params (w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        if i % 50 == 0:    
            print("Iteration: ", i) 
            print("Accuracy: ", get_accuracy(get_predictions (a2), Y))  
        return w1,b1,w2,b2

w1, b1, w2, b2 = gradient_descent(X_train,Y_train , 100, 0.1)    

<ipython-input-10-42b68ce8a0d1>:12: RuntimeWarning: overflow encountered in exp
  return np.exp(z) / np.sum(np.exp(z), axis = 0)
<ipython-input-10-42b68ce8a0d1>:12: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(z) / np.sum(np.exp(z), axis = 0)


Iteration:  0
[1 8 8 ... 1 8 8] [4 1 0 ... 2 3 8]
Accuracy:  0.09386440677966101
